## Retrieve and download data for Tableau Dashboard

### Connect to data sources and import libraries for modeling




In [ ]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#! pip install dmba

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import csv

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_predict
#from dmba import classificationSummary
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
import datetime as dt
from sklearn.metrics import accuracy_score, r2_score, recall_score, precision_score, f1_score

from imblearn.over_sampling import SMOTE

%matplotlib inline

# Turn off warnings for ease of viewing
import warnings
warnings.filterwarnings("ignore")

# Set option to see all columns in output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

### Connect to SQL database and bring in the transformed_data table

In [36]:
# Connect to the database
conn = sqlite3.connect('/content/drive/My Drive/MS-ADS Capstone Project/data/hr_case_study.db')

##### Query general_data table ##### 
# Create cursor object and query table
cursor = conn.cursor()
cursor.execute(""" SELECT N.EmployeeID,
                N.Department,
                N.BusinessTravel,
                N.EducationField,
                N.JobRole,
                CASE WHEN N.PerformanceRating = 3 THEN "Average" ELSE "Above Average" END AS "Performance Rating",
                CASE WHEN N.JobSatisfaction <= 2 THEN "Low" ELSE "High" END as "Job Satisfaction",
                P.attrition_prediction,
                P.attrition_probability

                FROM non_transformed_data as N 
                LEFT JOIN predictions_data as P
                ON N.EmployeeID = P.EmployeeID
                WHERE N.Attrition = 'No' """)

# Identify the column names for dataframe
col_names = [desc[0] for desc in cursor.description]

# Fetch all rows from the query and store in dataframe
rows = cursor.fetchall()
current_emp = pd.DataFrame(rows, columns = col_names)

# Close the cursor and connection objects
cursor.close()
conn.close()

In [39]:
# View the total active employee list
print(current_emp.shape)
current_emp.head()

(3699, 9)


,EmployeeID,Department,BusinessTravel,EducationField,JobRole,Performance Rating,Job Satisfaction,attrition_prediction,attrition_probability
0,1,Sales,Travel_Rarely,Life Sciences,Healthcare Representative,Average,High,0,0.002641
1,3,Research & Development,Travel_Frequently,Other,Sales Executive,Average,Low,0,0.041578
2,4,Research & Development,Non-Travel,Life Sciences,Human Resources,Average,High,0,0.001906
3,5,Research & Development,Travel_Rarely,Medical,Sales Executive,Average,Low,0,0.038741
4,6,Research & Development,Travel_Rarely,Life Sciences,Research Director,Average,Low,0,0.003617


In [41]:
# Write the output to Excel
current_emp.to_csv('/content/drive/My Drive/MS-ADS Capstone Project/data/current_employee_list_predictions.csv')